In [1]:
!nvidia-smi

Thu Feb  2 07:14:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    52W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **1. Get code and Data test**

In [5]:
# !git clone https://github.com/manhhv87/automatic-ecg-diagnosis.git
!git clone https://github.com/antonior92/ecg-age-prediction.git

Cloning into 'ecg-age-prediction'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 52 (delta 13), reused 13 (delta 13), pack-reused 30
Unpacking objects: 100% (52/52), 22.81 KiB | 141.00 KiB/s, done.


In [7]:
# !pip install -r /content/automatic-ecg-diagnosis/requirements.txt
!pip install -r /content/ecg-age-prediction/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://www.dropbox.com/s/p3vd3plcbu9sf1o/data.zip?dl=0 -O data.zip

In [ ]:
!unzip /content/data.zip -d /content/automatic-ecg-diagnosis

# **1. Get data from Google Cloude**

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
project_id = 'automatic-ecg-diagnosis'

In [3]:
!gcloud config set project {project_id}

Updated property [core/project].


In [4]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get -y -q update
!sudo apt-get -y -q install gcsfuse

deb http://packages.cloud.google.com/apt gcsfuse-focal main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1210  100  1210    0     0  80666      0 --:--:-- --:--:-- --:--:-- 80666
OK
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5,002 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [1,871 B]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:9 https://developer.download.nvidia.c

In [5]:
!mkdir -p /content/dataset

In [6]:
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 ecg12dataset /content/dataset

2023/02/02 07:17:32.174711 Start gcsfuse/0.41.12 (Go version go1.18.4) for app "" using mount point: /content/dataset
2023/02/02 07:17:32.187120 Opening GCS connection...
2023/02/02 07:17:34.514829 Mounting file system "ecg12dataset"...
2023/02/02 07:17:34.515177 File system has been successfully mounted.


# **3. Trainning**

In [7]:
# %cd /content/automatic-ecg-diagnosis
%cd /content/ecg-age-prediction

/content/ecg-age-prediction


## **3.1. Print model summary**

In [ ]:
!python model.py

## **3.2. Training model**

In [ ]:
# !python3 train.py \
#          --path_to_hdf5 "/content/dataset/exams_part0.hdf5" \
#          --path_to_csv "/content/dataset/exams.csv"

In [ ]:
!python3 train.py \
         --path_to_traces "/content/dataset/exams_part0.hdf5" \
         --path_to_csv "/content/dataset/exams.csv"

In [ ]:
import tensorflow as tf
print(tf. __version__)

2.2.0


In [9]:
import numpy as np
import h5py

In [10]:
hf = h5py.File('/content/dataset/exams_part0.hdf5', 'r')

In [11]:
hf.keys()

<KeysViewHDF5 ['exam_id', 'tracings']>

**input:** shape = (N, 4096, 12). The input tensor should contain the 4096 points of the ECG tracings sampled at 400Hz (i.e., a signal of approximately 10 seconds). Both in the training and in the test set, when the signal was not long enough, we filled the signal with zeros, so 4096 points were attained. The last dimension of the tensor contains points of the 12 different leads. The leads are ordered in the following order: {DI, DII, DIII, AVR, AVL, AVF, V1, V2, V3, V4, V5, V6}. All signal are represented as 32 bits floating point numbers at the scale 1e-4V: so if the signal is in V it should be multiplied by 1000 before feeding it to the neural network model.

In [12]:
n1 = hf['tracings']
n1

<HDF5 dataset "tracings": shape (20001, 4096, 12), type "<f4">

In [13]:
n1[0:5, :, :]

array([[[-4.8781034e-01, -2.6677129e-01,  2.2103906e-01, ...,
         -1.1737937e+00, -5.5640870e-01, -4.8781034e-01],
        [-4.8106578e-01, -2.6019633e-01,  2.2086945e-01, ...,
         -1.1674997e+00, -5.5151391e-01, -4.8232922e-01],
        [-4.7979322e-01, -2.5791711e-01,  2.2187607e-01, ...,
         -1.1599243e+00, -5.4229915e-01, -4.7729760e-01],
        ...,
        [-1.3724924e+00, -1.2811700e+00,  9.1322273e-02, ...,
         -8.7824506e-01, -5.0396240e-01,  8.5938675e-04],
        [-1.3667058e+00, -1.2758980e+00,  9.0807825e-02, ...,
         -8.6861408e-01, -4.9709535e-01,  1.6935735e-03],
        [-1.3614640e+00, -1.2694675e+00,  9.1996796e-02, ...,
         -8.5737538e-01, -4.8235169e-01,  1.0566236e-02]],

       [[ 4.7104187e+00,  3.4908926e+00, -1.2195258e+00, ...,
         -2.7439332e-01,  6.4787310e-01,  1.1128174e+00],
        [ 4.7069054e+00,  3.4883175e+00, -1.2185876e+00, ...,
         -2.7435562e-01,  6.4121950e-01,  1.1126217e+00],
        [ 4.7019477e+00, 

In [ ]:
import pandas as pd

In [ ]:
y = pd.read_csv("/content/dataset/exams.csv").values      

In [ ]:
y[:, 4:10]

In [ ]:
y1 = np.array(y)
y1.shape

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (confusion_matrix,
                             precision_score, recall_score, f1_score,
                             precision_recall_curve, average_precision_score)

In [ ]:
with h5py.File("/content/ecg-diagnosis-deep-learning/data/ecg_tracings.hdf5", "r") as f:
    x = np.array(f['tracings'])
print("Test dataset dimension:", x.shape)

In [ ]:
plt.title('ECG Signal')
plt.plot(x[12])
plt.show()

In [ ]:
y = np.load('dnn_output.npy')
print("Output dataset dimension for test data:", y.shape)